In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# common
import os
BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))

from datetime import datetime, timedelta
from io import BytesIO
from pathlib import Path
import json
from IPython.core.display import display, HTML

# network & protocol
import requests
from requests.exceptions import BaseHTTPError

# computational
import numpy as np
import pandas as pd
from scipy.stats import rankdata

import hydroeval as he

# graph
import plotly.express as px
import plotly.graph_objects as go

import bokeh.plotting as bp
from bokeh.models import tools as bmt, tickers
from bokeh.io import output_notebook, export_png

output_notebook()

# custom
import sys
sys.path.insert(0, f'{BASE_DIR}/scripts')
import stats



Loading BokehJS ...

In [3]:
start_date = '1990-01-01'
end_date = '2019-12-31'
date_tail = 'T00:00:00.000Z'
request_param = dict(sd=f'{start_date}{date_tail}', ed=f'{end_date}{date_tail}')
full_dates = pd.date_range(datetime.strptime(start_date, '%Y-%m-%d'),
                           datetime.strptime(end_date, '%Y-%m-%d'), freq='d')\
             .strftime('%Y-%m-%d').tolist()

In [4]:
_stations = pd.read_csv(f'{BASE_DIR}/other_exports/1990_2019_stations.csv')
stations_name = _stations.stationname.unique().tolist()
stations_name.sort()
print(stations_name)

['Battambang', 'Chaktomuk', 'Kg. Thmar', 'Koh Khel', 'Kompong Cham', 'Kompong Chen', 'Kompong Kdei', 'Kompong Thom', 'Kratie', 'Lumphat', 'Neak Luong', 'Phnom Penh Port', 'Prek Kdam', 'Siempang', 'Sisophon', 'Stung Treng', 'Voeun Sai']


In [5]:
date_tail = 'T00:00:00.000Z'
# stage_id: [ manual, telemetry ]
stations = [
    {
        'name': 'Battambang',
        'stage_id': ['cba718574b1f4244a4ae2ef58806732d', '8456239a6baf4221a744886c8cb6efd3'],
        'start_date': '1962-01-01',
        'end_date': '2020-12-31',
    },
    {
        'name': 'Chaktomuk',
        'stage_id': ['7f84c4d851a543d1a5eccc53947e0d3c', '2c070f6a900b41e7ad19bafef374f52d'],
        'start_date': '1959-01-01',
        'end_date': '2020-12-31',
    },
    {
        'name': 'Kompong_Thom',
        'stage_id': ['dae00e365f134b4bbfa6bf4162eb815c', 'aaebd95890ca49b9b5b5a82699159d6d'],
        'start_date': '1960-12-31',
        'end_date': '2020-12-30',
    },
    {
        'name': 'Kratie',
        'stage_id': ['fca7bee14dcf43a59263158f18a88509', '39b735a5de5c440cb0bc789fb9fdd544'],
        'start_date': '1923-12-31',
        'end_date': '2021-05-16',
    },
    {
        'name': 'Lumphat',
        'stage_id': ['93337ffe7838467289bf5b55401ee6f8', '6955b9585cc94524bcff184034e4afd3'],
        'start_date': '1964-12-31',
        'end_date': '2021-05-16',
    },
    {
        'name': 'Prek_Kdam',
        'stage_id': ['81e259eb15f64135834fe9e388ee7fbb', '5a7037f272c745d5bcafcc16a5c2f89c'],
        'start_date': '1960-10-12',
        'end_date': '2018-12-17',
    },
    {
        'name': 'Siempang',
        'stage_id': ['6a49fa1f6cb247d39b9e6c405205bfa3', '5794c0c5c5c045639e0efa1540466828'],
        'start_date': '1964-12-31',
        'end_date': '2012-12-30',
    },
    {
        'name': 'Stung_Treng',
        'stage_id': ['000e84564d1b425ebcc2c2292ed14061', '0a723f8802e44a6bb12c08dcf1816a83'],
        'start_date': '1909-12-31',
        'end_date': '2021-05-16',
    },
    {
        'name': 'Voeun_Sai',
        'stage_id': ['d177b0f2dae547ceb9bc7c23ef2970ae', 'e821b15baedd4af2a4ecde624b3bc976'],
        'start_date': '1964-12-31',
        'end_date': '2021-05-16',
    },
]

In [6]:
dfs = []

for station in stations:
    stage_id = station['stage_id']
    name = station['name']
    print(f'station {name}')
    
    table = {}
    table['name'] = name
    
    for idx, _id in enumerate(stage_id):
        df = pd.read_csv(f'{BASE_DIR}/mrc_download/{_id}.csv')
        if idx == 0:
            df.rename(columns = {'value': 'manual'}, inplace = True)
            table['manual'] = df
            
        else:
            df.rename(columns = {'value': 'telemetry'}, inplace = True)
            table['telemetry'] = df

    merged_df = pd.merge(table['manual'], table['telemetry'], how='inner', on='datetime')
    table['df'] = merged_df

    dfs.append(table)

station Battambang
station Chaktomuk
station Kompong_Thom
station Kratie
station Lumphat
station Prek_Kdam
station Siempang
station Stung_Treng
station Voeun_Sai


In [7]:
Path(f'{BASE_DIR}/plot_exports/manual_vs_telemetry').mkdir(parents=True, exist_ok=True)

for _df in dfs:
    df = _df['df']
    df.datetime = pd.to_datetime(df.datetime)

    idx = pd.date_range(df.datetime.min(), df.datetime.max())
    df.index = pd.DatetimeIndex(df.datetime)
    df = df.rename(index={'datetime': 'dateTimeIndex'})
    df = df.reindex(idx, fill_value=np.nan)
    df.datetime = pd.to_datetime(df.index)
    df.sort_index(ascending=True, inplace=True)

    station_name = _df['name']
    title = f'{station_name}: Manual Vs Telemetry'
    x_axis_label = 'datetime'
    y_axis_label = 'height (m)'

    # bokeh style
    TOOLS = 'pan,wheel_zoom,box_zoom,reset'
    hover_tool = bmt.HoverTool(
        tooltips=[
            ('name',   '$name'),
            ('date',   '$x{%F}'),
            ('height', '$y{0.00 a}'),
        ],
        formatters={
            '$x' : 'datetime',
        },
    )

    fig = bp.figure(title=title,
                    x_axis_label=x_axis_label,
                    x_axis_type='datetime',
                    y_axis_label=y_axis_label,
                    plot_height=300,
                    tools=TOOLS,
                   )
    fig.add_tools(hover_tool)
    fig.sizing_mode = 'scale_width'
    fig.xaxis.ticker.desired_num_ticks = len(np.unique(df.datetime.dt.year))

    fig.line(df.datetime, df.manual, name='manual', line_width=2, line_color='red', legend_label='manual')
    fig.line(df.datetime, df.telemetry, name='telemetry', line_width=2, line_color='blue', legend_label='telemetry')
    fig.legend.click_policy='hide'

    bp.show(fig)
    export_png(fig, filename=f'{BASE_DIR}/plot_exports/manual_vs_telemetry/{station_name}.png')
    
    # plotly style
#     scatter = go.Scattergl(x=df.datetime,
#                            y=df.manual,
#                            mode='lines',
#                            name='manual',
#                            marker=dict(color='red')
#                           )
    
#     scatter2 = go.Scattergl(x=df.datetime,
#                             y=df.telemetry,
#                             mode='lines',
#                             name='telemetry',
#                             marker=dict(color='blue')
#                            )

#     layout = go.Layout(title=title,
#                        xaxis=dict(title=x_axis_label),
#                        yaxis=dict(title=y_axis_label),
#                        showlegend=True)

#     chart_obj = go.Figure(data=[scatter, scatter2], layout=layout)

#     chart_obj.show()

In [11]:
df_dump = pd.DataFrame(columns = ['station', 'rmse', 'pr', 'nse', 'pbias', 'rsr'])

for _df in dfs:
    station = _df['name']

    df = _df['df']
    df.sort_index(ascending=True, inplace=True)

    manual = df.manual
    telemetry = df.telemetry

    # mean error
    me = stats.me(manual, telemetry)

    # mean absolute error
    mae = stats.mae(manual, telemetry)

    # mean squared error
    mse = stats.mse(manual, telemetry)

    # root mean squared error
    rmse = stats.rmse(manual, telemetry)

    # pearson's correlation coefficient
    pr = stats.pearson_cc(manual, telemetry)

    # nse
    nse = stats.nse(manual, telemetry)

    # pbias
    pbias = stats.pbias(manual, telemetry)

    # rsr
    rsr = stats.rsr(manual, telemetry)

    dump_record = {
        'station': station,
#         'me': me,
#         'mae': mae,
#         'mse': mse,
        'rmse': rmse,
        'pr': pr,
        'nse': nse,
        'pbias': pbias,
        'rsr': rsr,
    }
    df_dump = df_dump.append(dump_record, ignore_index=True)

display(HTML(df_dump.to_html()))

,station,rmse,pr,nse,pbias,rsr
0,Battambang,0.277821,0.987855,0.975764,-0.151410,0.155682
1,Chaktomuk,0.118016,0.999008,0.997968,-0.176584,0.045081
2,Kompong_Thom,0.065789,0.999726,0.999445,-0.077060,0.023554
3,Kratie,0.815538,0.979466,0.959193,0.187137,0.202009
4,Lumphat,0.378411,0.984874,0.969871,-0.480572,0.173583
5,Prek_Kdam,0.832381,0.948015,0.894977,-3.690282,0.324592
6,Siempang,0.678311,0.959741,0.919904,2.467652,0.283194
7,Stung_Treng,0.085070,0.999375,0.998545,-0.441247,0.038151
8,Voeun_Sai,0.498124,0.927030,0.854236,2.269794,0.382565
